In [1]:
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et
import pandas as pd
import urllib.request
from shapely.geometry import Point, shape
import json
import math

In [2]:
def get_haversine_distance(point_1, point_2):
    """
    Calculate the distance between any 2 points on earth given as [lon, lat]
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [point_1[0], point_1[1], 
                                                point_2[0], point_2[1]])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371000 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [3]:
city='Detroit'
SIM_AREA_PATH='./cities/'+city+'/clean/table_area.geojson'
SIM_NETWORK_PATH_ROOT='./cities/'+city+'/clean/'
FW_PATH='./cities/'+city+'/clean/fw_result.json'
INT_NET_LIST_PATH='./cities/'+city+'/clean/internal_net_list.json'
INT_NET_NODE_LLS_PATH='./cities/'+city+'/clean/internal_net_node_lls.json'

sim_area=json.load(open(SIM_AREA_PATH))

full_area=[shape(f['geometry']) for f in sim_area['features']]
bounds=[shp.bounds for shp in full_area]
boundsAll=[min([b[0] for b in bounds]), #W
               min([b[1] for b in bounds]), #S
               max([b[2] for b in bounds]), #E
               max([b[3] for b in bounds])] #N

In [4]:
overpass_node_query='https://lz4.overpass-api.de/api/interpreter?data=[out:json][bbox];node;out;&bbox='+','.join([str(b) for b in boundsAll])
overpass_way_query='https://lz4.overpass-api.de/api/interpreter?data=[out:json][bbox];way;out;&bbox='+','.join([str(b) for b in boundsAll])

with urllib.request.urlopen(overpass_node_query) as url:
    overpass_nodes=json.loads(url.read().decode())
with urllib.request.urlopen(overpass_way_query) as url:
    overpass_ways=json.loads(url.read().decode())

In [5]:
nodes=overpass_nodes['elements']

In [6]:
hw_ways=[]
for way in overpass_ways['elements']:
    if 'tags' in way:
        if 'highway' in way['tags']:
            hw_ways.append(way)

In [7]:
edges_types=set([way['tags']['highway'] for way in hw_ways])
edges_types

{'cycleway',
 'footway',
 'motorway',
 'motorway_link',
 'path',
 'pedestrian',
 'primary',
 'primary_link',
 'proposed',
 'residential',
 'secondary',
 'secondary_link',
 'service',
 'steps',
 'tertiary',
 'tertiary_link',
 'track',
 'trunk_link',
 'unclassified'}

In [8]:
drive_types=[t for t in edges_types if t not in ['footway', 'path', 'steps', 'pedestrian', 'track']]
cycle_types=[t for t in edges_types if t not in [ 'steps']]
pt_types=edges_types
walk_types=edges_types

In [9]:
def osm_to_geojson(ways, node_id_to_lon_lat, included_types, area, net_type):
    features=[]
    for way in ways:
        if way['tags']['highway'] in included_types:
            node_coords = []
            for node_id in way['nodes']:
                if node_id in node_id_to_lon_lat:
                    node_coords.append(node_id_to_lon_lat[node_id])
            in_sim_area=False
            for nc in node_coords:
                for feat in sim_area['features']:
                    if shape(feat['geometry']).contains(Point(nc)):
                        in_sim_area=True
            if ((in_sim_area) and (len(node_coords)>1)):    
                features.append({"type": "Feature",
                                 "properties": {'edge_type': net_type},
                                 'geometry': {
                                         "type": "LineString",
                                         'coordinates':node_coords},
                                 })
    geo= {"type": "FeatureCollection",
            "crs": { "type": "name", "properties": { "name": "epsg:4326" } },
            "features": features}
    return geo

In [10]:
node_id_to_lon_lat={}
for node in nodes:
    node_id_to_lon_lat[node['id']]=[node['lon'], node['lat']]

In [11]:
drive_net_geo=osm_to_geojson(hw_ways, node_id_to_lon_lat, drive_types, sim_area,'driving')
cycle_net_geo=osm_to_geojson(hw_ways, node_id_to_lon_lat, cycle_types, sim_area,'cycling')
walk_net_geo=osm_to_geojson(hw_ways, node_id_to_lon_lat, walk_types, sim_area,'walking')
pt_net_geo=osm_to_geojson(hw_ways, node_id_to_lon_lat, pt_types, sim_area,'pt')

In [12]:
json.dump(drive_net_geo, open(SIM_NETWORK_PATH_ROOT+'driving_net_simple.geojson', 'w'))
json.dump(walk_net_geo, open(SIM_NETWORK_PATH_ROOT+'walking_net_simple.geojson', 'w'))
json.dump(pt_net_geo, open(SIM_NETWORK_PATH_ROOT+'pt_net_simple.geojson', 'w'))
json.dump(cycle_net_geo, open(SIM_NETWORK_PATH_ROOT+'cycling_net_simple.geojson', 'w'))